In [1]:
# Importing libraries
import pyfabricops as pf
pf.set_auth_provider('env') 

import logging
pf.enable_notebook_logging() 

In [2]:
# Reading SQL on premisses payloads
import json

connections_path = '../connections.json'

with open(connections_path, 'r', encoding='utf-8') as f:
    connections = json.load(f)

sql_on_premises_connections = connections['sql_on_premises_connections']

print(json.dumps(sql_on_premises_connections, indent=4))

{
    "AdventureWorksDW2022-PRD": {
        "connection_id": "",
        "display_name": "AdventureWorksDW2022-PRD",
        "gateway_id": "7a27d5f4-081f-446c-90dd-031835787c0c",
        "server": "PEZZOTT",
        "database": "AdventureWorksDW2022-PRD",
        "credential_type": "Basic",
        "privacy_level": "Organizational",
        "connection_encryption": "NotEncrypted",
        "connection_type": "SQL",
        "roles": [
            {
                "user_uuid": "9322eb4a-4132-4bd1-8df1-5cd3d1d2400b",
                "user_type": "User",
                "role": "Owner"
            },
            {
                "user_uuid": "b025341a-965a-4c35-b12e-2af63e5eb58f",
                "user_type": "User",
                "role": "User"
            },
            {
                "user_uuid": "cbe83b10-3b7c-4479-8168-e5281fabf7ea",
                "user_type": "Group",
                "role": "UserWithReshare"
            }
        ]
    },
    "AdventureWorksDW2022-DEV": {
  

In [3]:
# Loading username and password for the connection:
from dotenv import load_dotenv
import os

load_dotenv()
username = os.getenv('DATABASE_USERNAME')
password = os.getenv('DATABASE_PASSWORD')

In [4]:
for connection_key, connection_data in sql_on_premises_connections.items():
    
    # Retrieving connection details
    display_name = connection_data['display_name']
    gateway_id = connection_data['gateway_id']
    server = connection_data['server']
    database = connection_data['database']
    roles = connection_data['roles']
    

    # Creating SQL on premises connection
    print(f"Creating SQL on premises connection: {display_name}...") 
    connection_created = pf.create_sql_on_premises_connection(
        display_name=display_name,
        gateway_id=gateway_id,
        server = server,
        database = database,
        username=username,
        password=password
    )

    # Extracting connection ID
    connection_id = connection_created['id']
    print(f"Connection created with ID: {connection_id}")

    # Assigning roles to the connection
    for role in roles:
        # Assigning role to the connection
        print(f"Assigning role {role['role']} to user {role['user_uuid']} ({role['user_type']}) for connection {display_name}...")
        pf.add_connection_role_assignment(
            connection=connection_id,
            user_uuid=role['user_uuid'],
            user_type=role['user_type'],
            role=role['role'],
        )

    # Storing connection ID in the connections dictionary
    connections['sql_on_premises_connections'][display_name]['connection_id'] = connection_id
    print(f"Connection {display_name} created and roles assigned successfully.\n")

print(json.dumps(connections, indent=4))

Creating SQL on premises connection: AdventureWorksDW2022-PRD...
Connection created with ID: 25c39679-c050-4b46-a7e3-6d2915a9542a
Assigning role Owner to user 9322eb4a-4132-4bd1-8df1-5cd3d1d2400b (User) for connection AdventureWorksDW2022-PRD...
Assigning role User to user b025341a-965a-4c35-b12e-2af63e5eb58f (User) for connection AdventureWorksDW2022-PRD...
Assigning role UserWithReshare to user cbe83b10-3b7c-4479-8168-e5281fabf7ea (Group) for connection AdventureWorksDW2022-PRD...
Connection AdventureWorksDW2022-PRD created and roles assigned successfully.

Creating SQL on premises connection: AdventureWorksDW2022-DEV...
Connection created with ID: 2411a00d-8722-46cc-abef-7298906bd27d
Assigning role Owner to user 9322eb4a-4132-4bd1-8df1-5cd3d1d2400b (User) for connection AdventureWorksDW2022-DEV...
Assigning role User to user b025341a-965a-4c35-b12e-2af63e5eb58f (User) for connection AdventureWorksDW2022-DEV...
Assigning role UserWithReshare to user cbe83b10-3b7c-4479-8168-e5281fabf7

In [5]:
# Saving updated connections to the file
with open(connections_path, 'w', encoding='utf-8') as f:
    json.dump(connections, f, indent=4)